<a href="https://colab.research.google.com/github/danilka4/chairdelure/blob/main/notebooks/LoRa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/tloen/alpaca-lora.git

Cloning into 'alpaca-lora'...
remote: Enumerating objects: 607, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 607 (delta 28), reused 34 (delta 19), pack-reused 556
Receiving objects: 100% (607/607), 27.78 MiB | 7.39 MiB/s, done.
Resolving deltas: 100% (360/360), done.


# Alpaca LLaMa 7B LoRa

In [ ]:
%cd alpaca-lora/

/content/alpaca-lora


In [ ]:
!pip install -q datasets loralib sentencepiece

!pip uninstall transformers
!pip install -q git+https://github.com/zphang/transformers@c3dc391
#!pip install -q git+https://github.com/zphang/transformers
# !pip install git+https://github.com/zphang/transformers.git@llama_push

!pip install -q git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 449.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82

In [ ]:
!pip install bitsandbytes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 16.5 MB/s eta 0:00:00


## Data Check

In [ ]:
from datasets import load_dataset
from transformers import LLaMATokenizer


tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf", add_eos_token=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

data = load_dataset("json", data_files="daniel_train.json")


def generate_prompt(data_point):
    # sorry about the formatting disaster gotta move fast
    return f"""{data_point["input"]} {data_point["output"]}"""
    if data_point["instruction"]:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{data_point["instruction"]}

### Input:
{data_point["input"]}

### Response:
{data_point["output"]}"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{data_point["instruction"]}

### Response:
{data_point["output"]}"""


data = data.map(lambda data_point: {"prompt": tokenizer(generate_prompt(data_point))})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-7b721a249ab1af33/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/39784 [00:00<?, ? examples/s]

## Fine Tuning

In [ ]:
import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoConfig, LLaMAForCausalLM, LLaMATokenizer
from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('//172.28.0.1'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

In [ ]:
# Setting for A100 - For 3090 
MICRO_BATCH_SIZE = 8  # change to 4 for 3090
BATCH_SIZE = 256
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
EPOCHS = 3  # paper uses 3
LEARNING_RATE = 2e-5  # from the original paper
CUTOFF_LEN = 256  # 256 accounts for about 96% of the data
LORA_R = 4
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

In [ ]:
model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
tokenizer = LLaMATokenizer.from_pretrained(
    "decapoda-research/llama-7b-hf", add_eos_token=True
)

model = prepare_model_for_int8_training(model)


Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [ ]:
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token
data = load_dataset("json", data_files="daniel_train.json")


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
test = load_dataset("json", data_files="daniel_test.json")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-16fe66e741c72c91/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:

def generate_prompt(data_point):
    # sorry about the formatting disaster gotta move fast
    return f"""{data_point["input"]} {data_point["output"]}"""
    if data_point["input"]:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{data_point["instruction"]}
### Input:
{data_point["input"]}
### Response:
{data_point["output"]}"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{data_point["instruction"]}
### Response:
{data_point["output"]}"""


data = data.shuffle().map(
    lambda data_point: tokenizer(
        generate_prompt(data_point),
        truncation=True,
        max_length=CUTOFF_LEN,
        padding="max_length",
    )
)
test = test.shuffle().map(
    lambda data_point: tokenizer(
        generate_prompt(data_point),
        truncation=True,
        max_length=CUTOFF_LEN,
        padding="max_length",
    )
)


trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    # test_dataset = test,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        warmup_steps=100,
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=True,
        logging_steps=1,
        output_dir="lora-alpaca",
        save_total_limit=3,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train(resume_from_checkpoint=False)

model.save_pretrained("lora-daniel")

Map:   0%|          | 0/39784 [00:00<?, ? examples/s]

Map:   0%|          | 0/9947 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1,4.599400
2,4.771600
3,4.765900
4,4.715400
5,4.604500
6,4.665800
7,4.718900
8,4.826300
9,4.544800
10,4.651000


In [7]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub("danilka1/capstone_model", use_auth_token=True, create_pr=0)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/8.43M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/danilka1/capstone_model/commit/8084e3c6c8e3d7708f6fdeec45e0fc8858329151', commit_message='Upload model', commit_description='', oid='8084e3c6c8e3d7708f6fdeec45e0fc8858329151', pr_url=None, pr_revision=None, pr_num=None)

## Generation

In [2]:
!pip install -q datasets loralib sentencepiece

!pip uninstall transformers
!pip install -q git+https://github.com/zphang/transformers@c3dc391
#!pip install -q git+https://github.com/zphang/transformers
# !pip install git+https://github.com/zphang/transformers.git@llama_push

!pip install -q git+https://github.com/huggingface/peft.git
!pip install bitsandbytes

Found existing installation: transformers 4.27.0.dev0
Uninstalling transformers-4.27.0.dev0:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.10/dist-packages/transformers-4.27.0.dev0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)? ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/uninstall.py", line 105, in run
    uninstall_pathset = req.uninstall(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/req/req_install.py", line 680, in uninstall
    uninstalled_pathset.remove(auto_confirm, verbose)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/req/req_uninstall.py", line 375, in remove
    if auto_confirm or self._allowed_to_proceed(verbose):
  File "/usr/local/

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig

tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf",
                                           device_map={"":0})

model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map={"":0},
)



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('//172.28.0.1'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [4]:
model = PeftModel.from_pretrained(model, "danilka1/capstone_model", device_map={"":0})

In [10]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig
from tqdm import tqdm
# tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf",
#                                            device_map={"":0})
# model = PeftModel.from_pretrained(model, "danilka1/capstone_model", device_map={"":0})

# model = LLaMAForCausalLM.from_pretrained(
#     "decapoda-research/llama-7b-hf",
#     load_in_8bit=True,
#     device_map={"":0},
# )

def generate_word_recommendations(prompt, extra_tokens=5):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    
    # Check if input_ids is empty or contains only special tokens
    if input_ids.size(1) == 0 or all(token in tokenizer.all_special_ids for token in input_ids[0]):
        return ''

    max_length = len(input_ids[0]) + extra_tokens

    # Create attention_mask
    attention_mask = input_ids.ne(tokenizer.eos_token_id).long()
    output = model.generate(
        input_ids=input_ids,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=False,
        top_k=20,
        top_p=1.0,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id,  # Set pad_token_id to eos_token_id
        attention_mask=attention_mask,  # Pass attention_mask
    )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

def get_cases(data):
    cases = []
    for message in data:
        words = message.split()
        for i in range(1,len(words)):
            context = ' '.join(words[:i])
            word = words[i]
            cases.append((context, word))
    return cases

def evaluate_accuracy(test_file_path):
    total_count = 0
    correct_count = 0

    with open(test_file_path, 'r') as test_file:
        lines = test_file.readlines()
        cases = get_cases(lines)
        total_lines = len(cases)
        
        for idx, (prompt, expected_word) in enumerate((cases)):
            prompt = prompt.replace('-',' ').replace('_',' ').replace('\n',' ').replace(' ,', ',').replace(' .', '.').replace(' ?', '?').replace(' !', '!')
            print('Prompt: ', prompt)
            print('Expected Word: ', expected_word)
            p_len = len(prompt.split(' '))
            print('Prompt Length: ', p_len)
            generated_text = generate_word_recommendations(prompt)
            print('Generated Text: ', generated_text)
            generated_text = generated_text.replace('-',' ').replace('_',' ').replace('\n',' ').split()
            try:
                generated_word = generated_text[p_len]
            except IndexError:
                generated_word = ''

            print('Generated Words: ', generated_word)
            
            total_count += 1
            if expected_word == generated_word:
                correct_count += 1
            
            progress = (idx + 1) / total_lines * 100
            print(f"\r Progress: {progress:.2f}% | {idx+1}/{total_lines} | Correct: {correct_count} / {total_count}")

    accuracy = correct_count / total_count
    return accuracy

test_file_path = "./daniel_test.txt"
print('Evaluating Accuracy')
accuracy = evaluate_accuracy(test_file_path)
print(f"\nAccuracy: {accuracy:.2%}")

Evaluating Accuracy
Prompt:  "1
Expected Word:  big
Prompt Length:  1


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1374: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Streaming output truncated to the last 5000 lines.
Expected Word:  return
Prompt Length:  8
Generated Text:   ```c // try to take a turn and print the board ```
Generated Words:  print
 Progress: 26.46% | 9197/34757 | Correct: 2243 / 9197
Prompt:  ```c // try to take a turn and return
Expected Word:  true
Prompt Length:  9
Generated Text:   ```c // try to take a turn and return None```
Generated Words:  None```
 Progress: 26.46% | 9198/34757 | Correct: 2243 / 9198
Prompt:  ```c // try to take a turn and return true
Expected Word:  if
Prompt Length:  10
Generated Text:   ```c // try to take a turn and return true if it's a
Generated Words:  if
 Progress: 26.47% | 9199/34757 | Correct: 2244 / 9199
Prompt:  ```c // try to take a turn and return true if
Expected Word:  the
Prompt Length:  11
Generated Text:   ```c // try to take a turn and return true if it's a valid
Generated Words:  it's
 Progress: 26.47% | 9200/34757 | Correct: 2244 / 9200
Prompt:  ```c // try to take a turn and return 

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 88>:88                                                                            │
│ in evaluate_accuracy:66                                                                          │
│ in generate_word_recommendations:25                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/peft/peft_model.py:731 in generate                       │
│                                                                                                  │
│    728 │   │   self.base_model.prepare_inputs_for_generation = self.prepare_inputs_for_generati  │
│    729 │   │   try:                                                                              │
│    730 │   │   │   if not isinstance(peft_config, PromptLearningConfig):                         │
│ ❱  731 │   │   │   │   outputs = self.base_model.generate(**kwargs)                              │
│    732 │   │   │   else:                                                                         │
│    733 │   │   │   │   if "input_ids" not in kwargs:                                             │
│    734 │   │   │   │   │   raise ValueError("input_ids must be provided for Peft model generati  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py:115 in decorate_context       │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1406 in generate        │
│                                                                                                  │
│   1403 │   │   │   │   )                                                                         │
│   1404 │   │   │                                                                                 │
│   1405 │   │   │   # 11. run greedy search                                                       │
│ ❱ 1406 │   │   │   return self.greedy_search(                                                    │
│   1407 │   │   │   │   input_ids,                                                                │
│   1408 │   │   │   │   logits_processor=logits_processor,                                        │
│   1409 │   │   │   │   stopping_criteria=stopping_criteria,                                      │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2201 in greedy_search   │
│                                                                                                  │
│   2198 │   │   │   model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)  │
│   2199 │   │   │                                                                                 │
│   2200 │   │   │   # forward pass to get next token        

In [23]:
print("hi", end="\r")

In [ ]:
!git clone https://github.com/danilka4/chairdelure.git

Cloning into 'chairdelure'...
remote: Enumerating objects: 142, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 142 (delta 55), reused 113 (delta 32), pack-reused 0
Receiving objects: 100% (142/142), 4.26 MiB | 19.75 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [ ]:
!cp -r chairdelure/templates templates

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))
!python -m http.server 5000
     

https://uliinok60f-496ff2e9c6d22116-5000-colab.googleusercontent.com/
Serving HTTP on 0.0.0.0 port 5000 (http://0.0.0.0:5000/) ...

Keyboard interrupt received, exiting.
^C


In [ ]:
from google.colab.output import eval_js
import flask
app = flask.Flask(__name__)

# Run input on model
@app.route('/model', methods=['POST'])
def model():
    try:
        # Get form data
        selected_model = flask.request.form.get('model')
        text = flask.request.form.get('text')

        # Run model
        if selected_model == 'molly':
            # output = molly(text)
            output = "Molly model not loaded"
        elif selected_model == 'daniel':
            output = daniel(text)
        elif selected_model == 'chris':
            output = "Chris model not loaded"
        else:
            output = 'Error: Invalid model'
        # Return output as JSON
        return flask.jsonify({'output': output})

    except Exception as e:
        return flask.jsonify({'error': str(e)})

# Show about page
@app.route('/about')
def about():
    return flask.render_template('about.html')

@app.route('/')
def hello():
    return flask.render_template('index.html')

print('App will be at')
print(eval_js("google.colab.kernel.proxyPort(5000)"))
app.run()

App will be at
https://uliinok60f-496ff2e9c6d22116-5000-colab.googleusercontent.com/
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:


!pip install discord
import os
from re import split

import discord
from discord.ext import commands

import nest_asyncio
nest_asyncio.apply()

class Commands(commands.Cog):
    def __init__(self, bot):
        self.bot = bot

    @commands.command()
    async def ping(self, ctx):
        """Returns bot latency to client"""
        return await ctx.send(
            f"Pong! Latency: {self.bot.ws.latency * 1000:.4f}ms"
        )
    
    @commands.command()
    async def molly(self, ctx, *, message: str):
        output = molly(message)
        await ctx.send(output)

    @commands.command()
    async def daniel(self, ctx, *, message: str):
        output = daniel(message)
        await ctx.send(output)
    
    @commands.command()
    async def chris(self, ctx, *, message: str):
        output = chris(message)
        await ctx.send(output)

async def setup(bot):
    await bot.add_cog(Commands(bot))

class DiscordGPT(commands.Bot):
    def __init__(self):
        """Sets permissions and initializes bot object"""
        intents = discord.Intents.all()
        super().__init__(command_prefix="*", intents=intents)

    async def load_extensions(self):
        """Loads commands (called cogs or extensions)"""
        await self.add_cog(Commands(self))
        print("Successfully loaded commands.")

    async def on_connect(self):
        await self.load_extensions()

    async def on_ready(self):
        """Bot startup"""
        print(f"{self.user.name} has connected to Discord!")

    async def on_command_error(self, ctx, error):
        """Sends error message if caught, else raise error to terminal"""
        if isinstance(error, commands.CommandNotFound):
            return await ctx.send(
                f"Command not found: `{split(r'[ *]+', ctx.message.content)[1]}`"
            )
        raise error


def main():
    # Pulls bot token from environment 
    TOKEN = "MTA4NzQ0Mjk4OTkwMDYyODA3MA.G9COYk.lAy8tZu2OUeZZX3_3U2D-m2Nk4yOS_DnQxfnnw"

    bot = DiscordGPT()
    # Spins up bot instance
    bot.run(TOKEN)

main()
     


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 22.7 MB/s eta 0:00:00


2023-04-18 02:38:18 INFO     discord.client logging in using static token
INFO:discord.client:logging in using static token


LoginFailure: ignored